# Part 1 -- Setup & Collection

Pull Tweets from the Twitter API and collect **all Tweets from 30 tech thought leaders and news outlets**.

### Load lib codes:

In [1]:
!pwd

/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/ipynb


In [2]:
from os import chdir
chdir('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/')

from lib import *
# suppress_warnings()
from lib.twitter_keys import my_keys

In [3]:
!pip install pymongo
import pymongo

    100% |████████████████████████████████| 368kB 2.5MB/s ta 0:00:01
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
cli = pymongo.MongoClient(host='35.163.253.8')

In [5]:
# Instantiates when you put data in 
task_collection = cli.twitter_db.task_collection
completed_collection = cli.twitter_db.completed_collection
# tweets_collection = cli.twitter_db.tweets_collection
cli.twitter_db.collection_names()

['completed_collection', 'task_collection']

In [6]:
print(task_collection.count())
print(completed_collection.count())

30
87289


In [10]:
87289.0/90000.0
#96.98% of possible tweets achieved - in 02:12 minutes!

0.9698777777777777

In [65]:
87258.0/90000.0
#96.95% of possible tweets achieved - in 04:26 minutes

0.9695333333333334

In [52]:
87244.0/90000.0
#96.93% of possible tweets achieved

0.9693777777777778

In [23]:
87222.0/90000.0
#96.91% of possible tweets achieved

0.9691333333333333

In [13]:
curs = task_collection.find()
list_of_docs = []
for i in range(curs.count()):
    list_of_docs.append(curs.next())
    df = pd.DataFrame(list_of_docs)
df

,_id,handle,status
0,5935926d2c7311008077b547,BarackObama,complete
1,5935927056753600d03faa4e,cnnbrk,complete
2,593592742c7311008077be92,BillGates,complete
3,5935927956753600d03fb700,realDonaldTrump,complete
4,5935927e2c7311008077cb1b,TheEconomist,complete
5,5935928356753600d03fc380,aplusk,complete
6,593592882c7311008077d7bc,HillaryClinton,complete
7,5935928d56753600d03fd010,TechCrunch,complete
8,593592912c7311008077e379,elonmusk,complete
9,5935929656753600d03fdca1,NewYorker,complete


In [7]:
tweets_df =  pd.DataFrame(list(completed_collection.find()))
tweets_df.sample(3)

,_id,text,timestamp,username
20201,5935928356753600d03fc3de,Watch SpaceX launch a recycled Dragon capsule ...,2017-06-03 18:07:56,TechCrunch
35151,5935929a2c7311008077ee53,"When it comes to your investments... ""The end...",2017-05-31 07:31:19,MarketWatch
81241,593592e256753600d04039e6,🛫 Heading to #SIC2017,2017-05-24 12:49:39,MarkYusko


### Try different vectorizers on non-cleaned data first:

#### Clean data:

In [8]:
def create_document_term_matrix(vectorizer, corpus):
    document_term_matrix = vectorizer.fit_transform(corpus)
    document_term_matrix = pd.DataFrame(document_term_matrix.toarray(),
                                        index=corpus,
                                        columns=vectorizer.get_feature_names ())
    return document_term_matrix

In [9]:
tweets_df['text']

0        On this National Gun Violence Awareness Day, l...
1        Forever grateful for the service and sacrifice...
2        Good to see my friend Prince Harry in London t...
3        Through faith, love, and resolve the character...
4        Our hearts go out to those killed and wounded ...
5        Excited to hear from Sierra, Imani, Filiz, and...
6        Happy Mother's Day to my love and partner on t...
7        We're rolling up our sleeves again, back where...
8        Well said, Jimmy. That's exactly why we fought...
9        RT @ObamaFoundation: It’s a beautiful day on t...
10       My heart goes out to the victims and their fam...
11       Chuck Berry rolled over everyone who came befo...
12       On International Women’s Day, @MichelleObama a...
13       RT @ObamaFoundation: Courage comes in many for...
14       Humbled to be recognized by a family with a le...
15         We asked. You answered. https://t.co/mAJvko6VqR
16       Happy Valentine’s Day, @michelleobama! Almost .

In [10]:
import re

In [11]:
def clean_text(text):
    text = re.sub("'", "", text)
    text = re.sub('(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{0,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', '', text)
    text = re.sub('[\W]', ' ', text)
    return text

In [12]:
tweets_df['cleaned_text'] = tweets_df['text'].apply(clean_text)

In [13]:
tweets_df

,_id,text,timestamp,username,cleaned_text
0,593592642c7311008077a8a5,"On this National Gun Violence Awareness Day, l...",2017-06-02 17:35:54,BarackObama,On this National Gun Violence Awareness Day l...
1,593592642c7311008077a8a6,Forever grateful for the service and sacrifice...,2017-05-29 13:09:16,BarackObama,Forever grateful for the service and sacrifice...
2,593592642c7311008077a8a7,Good to see my friend Prince Harry in London t...,2017-05-27 13:15:25,BarackObama,Good to see my friend Prince Harry in London t...
3,593592642c7311008077a8a8,"Through faith, love, and resolve the character...",2017-05-25 14:13:35,BarackObama,Through faith love and resolve the character...
4,593592642c7311008077a8a9,Our hearts go out to those killed and wounded ...,2017-05-23 16:56:14,BarackObama,Our hearts go out to those killed and wounded ...
5,593592642c7311008077a8aa,"Excited to hear from Sierra, Imani, Filiz, and...",2017-05-22 21:16:23,BarackObama,Excited to hear from Sierra Imani Filiz and...
6,593592642c7311008077a8ab,Happy Mother's Day to my love and partner on t...,2017-05-14 14:01:16,BarackObama,Happy Mothers Day to my love and partner on th...
7,593592642c7311008077a8ac,"We're rolling up our sleeves again, back where...",2017-05-03 19:42:16,BarackObama,Were rolling up our sleeves again back where ...
8,593592642c7311008077a8ad,"Well said, Jimmy. That's exactly why we fought...",2017-05-02 17:19:26,BarackObama,Well said Jimmy Thats exactly why we fought ...
9,593592642c7311008077a8ae,RT @ObamaFoundation: It’s a beautiful day on t...,2017-04-24 19:48:17,BarackObama,RT ObamaFoundation It s a beautiful day on t...


#### BAG OF WORDS: CountVectorizer

In [21]:
!pip install spacy

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
from spacy.en import English

ImportError: cannot import name 'where'

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [22]:
naive_vectorizer = CountVectorizer(min_df = 3)
document_term_matrix = create_document_term_matrix(naive_vectorizer, tweets_df['text'])
document_term_matrix.sample(5)
#this is a document term matrix. It just counts the # of times a word shows up:

MemoryError: 

### Do a PCA to determine number of components for SVD: